# Hadamard Test

Consider the observable $O$ and two generic quantum states $\ket{\psi}$ and $\ket{\phi}$. We want to calculate the quantity
$$
\braket{\psi | O | \psi}.
$$
where $O$ is a Pauli operator.

First of all we shall prepare the states $\ket{\psi}$ and $\ket{\phi}$ using a quantum circuit for each of them. So we  have
$$
\ket{\psi} = U_{\psi}\ket{0} \qquad \ket{\phi} = U_{\phi}\ket{0}
$$

Let's define an observable we want to use:
$$
O = X_1X_2
$$

Now we can evaluate the matrix element using the following fact:
$$
\bra{\psi}O\ket{\phi} = \bra{0}U_\psi^\dagger O U_\phi\ket{0}
$$
This is just an expectation value which can be solved with a simple Hadamard test. The probability to measure $0$ or $1$ in the ancilla qubit is

$$
P(0) = \frac{1}{2} \left[ I + \operatorname{Re} \bra{\psi} O \ket{\phi} \right]
$$

$$
P(1) = \frac{1}{2} \left[ I - \operatorname{Re} \bra{\psi} O \ket{\phi} \right]
$$

The difference between the probability of $0$ and $1$ gives 

$$
\braket{X} = P(0)-P(1) = \operatorname{Re} \braket{\psi | O | 
\phi}.
$$

Similarly, the imaginary part can be obtained from Y measurement
$$
\braket{Y} = \operatorname{Im} \braket{\psi | O | \phi}.
$$

Combining these results, the quantity $\braket{\psi | O | \psi}$ is obtained.

### Numerical result as a reference: 

In [1]:
from functools import reduce

import numpy as np

import cudaq

cudaq.set_target("nvidia")

num_qubits = 2


@cudaq.kernel
def psi(num_qubits: int):
    q = cudaq.qvector(num_qubits)
    h(q[1])


@cudaq.kernel
def phi(num_qubits: int):
    q = cudaq.qvector(num_qubits)
    x(q[0])


psi_state = cudaq.get_state(psi, num_qubits)
print("Psi state: ", np.array(psi_state))

phi_state = cudaq.get_state(phi, num_qubits)
print("Phi state: ", np.array(phi_state))

ham = cudaq.spin.x(0) * cudaq.spin.x(1)
ham_matrix = ham.to_matrix()
print("hamiltonian: ", np.array(ham_matrix), "\n")

num_exp_val = np.array(psi_state).conj() @ ham_matrix @ np.array(phi_state).T

print("Numerical expectation value: ", num_exp_val)

Psi state:  [0.70710677+0.j 0.        +0.j 0.70710677+0.j 0.        +0.j]
Phi state:  [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
hamiltonian:  [[0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]] 

Numerical expectation value:  (0.7071067690849304+0j)


### Using ``observe`` algorithmic primitive to compute the expectation value for ancilla qubits.

In [2]:
import cudaq

cudaq.set_target("nvidia")


@cudaq.kernel
def u_psi(q: cudaq.qview):
    h(q[1])


@cudaq.kernel
def u_phi(q: cudaq.qview):
    x(q[0])


@cudaq.kernel
def apply_pauli(q: cudaq.qview):
    x(q[0])
    x(q[1])


@cudaq.kernel
def kernel(num_qubits: int):
    ancilla = cudaq.qubit()
    q = cudaq.qvector(num_qubits)
    h(ancilla)
    cudaq.control(u_phi, ancilla, q)
    cudaq.control(apply_pauli, ancilla, q)
    cudaq.control(u_psi, ancilla, q)


num_qubits = 2
shots = 100000
x_0 = cudaq.spin.x(0)
y_0 = cudaq.spin.y(0)
results = cudaq.observe(kernel, [x_0, y_0], num_qubits, shots_count=shots)
exp_vals = np.array([result.expectation() for result in results])
std_errors = np.sqrt((1 - exp_vals**2) / shots)

print(f"QC result: {exp_vals[0]}+{exp_vals[1]}i ± {std_errors[0]}+{std_errors[1]}i")
print("Numerical result", num_exp_val)

QC result: 0.70458+-0.00489999999999996i ± 0.0022440299097828444+0.0031622396967971927i
Numerical result (0.7071067690849304+0j)
